### 1. Форматування

[The Associated Press Stylebook](https://www.amazon.com/Associated-Press-Stylebook-2017-Briefing/dp/0465093043/) - це посібник зі стилю, яким часто послуговуються журналісти по всьому світу. Він рекомендує такі правила форматування заголовків:
1. З великої літери потрібно писати слова довжиною 4 чи більше літер.
2. З великої літери потрібно писати перше і останнє слово заголовку, незалежно від частини мови.
3. З великої літери потрібно писати іменники, займенники, дієслова, прикметники, прислівники та підрядні сполучники.
4. Якщо слово написане через дефіс, велику літеру потрібно додати для кожної частинки слова (наприклад, правильно "Self-Reflection", а не "Self-reflection").
5. З маленької літери потрібно писати всі інші частини мови: артиклі/визначники, сурядні сполучники, прийменники, частки, вигуки.

**Завдання:**
1. напишіть програму, яка форматує заголовки за вказаними правилами
2. перевірте якість роботи програми на [валідаційній вибірці](data/headlines-test-set.json)
3. проженіть вашу програму на [корпусі заголовків з The Examiner](data/examiner-headlines.txt) і вирахуйте, скільки заголовків там відформатовано за правилами (скільки заголовків залишились незмінними після форматування)
4. збережіть програму та числові результати у директорії з вашим іменем

Якщо потрібно продебажити роботу програми, робіть це на [корпусі заголовків з The Examiner](data/examiner-headlines.txt), а не на валідаційній вибірці. Спробуйте досягти хоча б 90% якості на валідаційній вибірці. Якість рахуємо за повним збігом відформатованого заголовка.

Підказка: ваша програма повинна правильно розрізняти прийменники та підрядні сполучники. Наприклад, `Do as you want` => `Do As You Want` (бо "as" тут є сполучником), but `How to use a Macbook as a table` => `How to Use a Macbook as a Table` (бо "as" тут є прийменником).

In [16]:
import re
import json
import spacy
import en_core_web_sm

CCASE_PATTERN = re.compile(r'([A-Z]+[a-z0-9]*)+')

CAP_CASES = ['NOUN', 'PROPN', 'PRON', 'VERB', 'ADJ', 'ADV'] # 'SCONJ'
LOW_CASES = ['DET','ADP','CCONJ', 'INT', 'PART']

nlp = spacy.load('en_core_web_sm')

def is_first_word(t):
    i = 0
    w = t.doc[i]
    
    while not (w.is_alpha or w.is_upper or w.is_digit):
        i += 1
        w = t.doc[i]
    return t.i == i

def is_last_word(t):
    i = len(t.doc) - 1
    w = t.doc[i]
    
    while not (w.is_alpha or w.is_upper or w.is_digit):
        i -= 1
        w = t.doc[i]
    return t.i == i

def skip_capitalize(t):
    return (t.pos_ in CAP_CASES and t.is_upper) or \
           (t.pos_ in CAP_CASES and CCASE_PATTERN.match(t.text)) or \
           (t.pos_ == 'NUM' and t.dep_ == 'compound') or \
           t.text[0] == "'"

def need_capitalize(t): 
    return is_first_word(t) or \
           is_last_word(t) or \
           len(t.text) > 3 or \
           t.pos_ in CAP_CASES or \
           (t.pos_ == 'ADP' and t.dep_ == 'mark') or \
           (t.pos_ == 'DET' and (t.dep_ == 'poss' or t.dep_ == "appos")) or \
           (t.text == 'not') or \
           (t.pos_ == 'AUX') or \
           (t.pos_ == 'NUM') or \
           (t.pos_ == 'SCONJ' and t.dep_ == 'mark') or \
           ((t.i > 0) and (t.doc[t.i - 1].text == '-')) or \
           ((t.i > 0) and (t.doc[t.i - 1].text == "'"))

def format_heading(str):
    doc = nlp(str)
    result = []
    
    # import code; code.interact(local=dict(globals(), **locals()))

    for t in doc:
        # print(t.text, t.pos_, t.dep_, t.is_title, t.is_alpha, need_capitalize(t), t.head, t.is_left_punct, t.left_edge, t.right_edge)
    
        if skip_capitalize(t):
            result.append(t.text_with_ws)
        elif need_capitalize(t):
            result.append(t.text_with_ws.title())
        elif t.pos_ in LOW_CASES:
            result.append(t.text_with_ws.lower())
        else:
            result.append(t.text_with_ws)

    # print(''.join(result))
    return ''.join(result)

In [17]:
assert format_heading("Don't stop me.") == "Don't Stop Me."
assert format_heading("Do as you want") == "Do As You Want"
assert format_heading("How to use a Macbook as a table") == "How to Use a Macbook as a Table"
assert format_heading("Halep enters Rogers Cup final in straight sets win over Errani") == "Halep Enters Rogers Cup Final in Straight Sets Win Over Errani"
assert format_heading("SF Beer Week 2013: what's for dinner (part 2)") == "SF Beer Week 2013: What's for Dinner (Part 2)"
assert format_heading("San Jose's Evergreen College program is part of Latino graduation increase") == "San Jose's Evergreen College Program Is Part of Latino Graduation Increase"
assert format_heading("Border insecurity and illegal-migration renders 'gang of eight' deal DOA") == "Border Insecurity and Illegal-Migration Renders 'Gang of Eight' Deal DOA"
assert format_heading("Courtney J. Walker named SEC Freshman of the Week in women's basketball (Photos)") == "Courtney J. Walker Named SEC Freshman of the Week in Women's Basketball (Photos)"
assert format_heading("'Minecraft: Xbox 360 Edition' mash-up pack free trial and suggestions discussed") == "'Minecraft: Xbox 360 Edition' Mash-Up Pack Free Trial and Suggestions Discussed"
assert format_heading("Review for Sarah MacLean's, Eleven Scandals to Start to Win a Duke's Heart") == "Review for Sarah MacLean's, Eleven Scandals to Start to Win a Duke's Heart"
assert format_heading("First set photos from the 'If I Stay' movie adaptation revealed") == "First Set Photos From the 'If I Stay' Movie Adaptation Revealed"
assert format_heading("The phantoms of St. Mary's") == "The Phantoms of St. Mary's"
assert format_heading("Talladega turmoil could spell trouble for NASCAR's Chase field") == "Talladega Turmoil Could Spell Trouble for NASCAR's Chase Field"
assert format_heading("2011-2012 NHL team preview: Detroit Red Wings") == "2011-2012 NHL Team Preview: Detroit Red Wings"
assert format_heading("Google science fair to encourage STEM learning") == "Google Science Fair to Encourage STEM Learning"
assert format_heading("Joshua Ledet begs, Phillip Phillips is a zombie on 'American Idol' 2012 top 5") == "Joshua Ledet Begs, Phillip Phillips Is a Zombie on 'American Idol' 2012 Top 5"
assert format_heading("USWNT to host Mexico at Red Bulls Arena June 5") == "USWNT to Host Mexico at Red Bulls Arena June 5"
assert format_heading("MasterCard faces £19bn lawsuit over claims it ripped off shoppers") == "MasterCard Faces £19bn Lawsuit Over Claims It Ripped off Shoppers"
assert format_heading("Credit: Make sure a new year hangover is not on the cards") == "Credit: Make Sure a New Year Hangover Is Not on the Cards"

In [14]:
exam_set = '../../../tasks/02-structural-linguistics/data/examiner-headlines.txt'

with open(exam_set) as headlines:
    total = 0
    changed = 0
    for h in headlines:
        total += 1
        if format_heading(h) != h:
            changed += 1
    print("Total Headlines: {}".format(total))
    print("Changed Headlines: {} ({:.2f}%)".format(changed, (changed * 100) / total))

Total Headlines: 5000
Changed Headlines: 4412 (88.24%)


In [18]:
# Final Test

test_set = '../../../tasks/02-structural-linguistics/data/headlines-test-set.json'

with open(test_set) as f:
    data = json.load(f)
    counter = 0
    match = 0

    for test, result in data:
        counter += 1
        if format_heading(test) == result:
            match += 1

match / counter

0.92